In [1]:
import numpy as np
import cmath
import matplotlib.pyplot as plt
from scipy.signal import find_peaks as fp
from mpmath import ellipk,log
from scipy.special import gamma as GAMMA
from scipy.special import expit
import os 
#from mpl_toolkits import mplot3d
from scipy.optimize import fmin_cg, fmin_tnc, curve_fit, newton, bisect, fsolve
from SourcesTBG import *
from MoreSources import *
from scipy.misc import derivative as scider
import pandas as pd
from scipy.linalg import eigh,eig,eigvalsh
from functools import partial
import seaborn as sns
from FeynmanHellman import *
from saddlefinder import *
import csv
from Decorators import *
from YuanDos import *
from DosHelper import *
TO_DEGREE = 1.0/0.018326
IN_RADIAN = np.pi/180
vFpar = 4.31074647887324
wpar=0.11
BANDS = 8

# Explanation: This next cell makes the DoS at all angles away from the magic angle by stitching artificially the cone DoS and the Log DoS. 

#### The cone dos is $A E$, and the vhs dos is $P \log\left|\frac{B-E_v}{E-E_v}\right|$. The matching scale is $E = E_*$, which has been arbitrarily chosen as $0.8 E_v$.  
#### With $x = e^{-\frac{E}{E_*}}$, the DoS is stitched by:
$\left(x\right) A E + \left(1-x\right) P \log\left|\frac{B-E_v}{E-E_v}\right|$ 

#### The version DoSpass1 used the smoothened step function, but this new version is found to be smoother by plotting the gradient. 

#### In the Log dos, the cutoff scale (BW-Ev) is chosen so that the logarithmic tail ends exactly at the band edge of 3eV. 

#### The numbers A(slope of the cone Dos) is an analytical relation as a function of twist angle, the number P is determined from the numbers alpha and beta away from the magic angle. These are returned by the function ret_alpha_beta for each twist angle. 

### Note: this is less accurate very close to the magic angle.

In [ ]:
BASE_PATH = "C:/Users/Aravi/OneDrive/Documents/DoS_Pass2/"
if not os.path.exists(BASE_PATH):
     raise(Exception("CHECK THE BASE PATH"))

BW = 3.0 #consider a rough estimate of the bandwidth to be 3eV
Eset = BW*np.logspace(-12,0,13*10000)
thetaiset = [0.8,0.85,0.9,0.95,1.0,1.1,1.15,1.2,1.25,1.4,1.5,1.6,1.8,2.0]
TOWRITE_X = np.concatenate((-np.flip(Eset),Eset))
for ival, thetai in enumerate(thetaiset):
    funcreturned = ret_alpha_beta_Ev(thetai)
    alpha, beta = funcreturned[0]  #Note NO NEED to multiply 0.5 to the values in the returned alphabeta
    Ev = funcreturned[1]
    thetarad = thetai * IN_RADIAN
    A = DiracConeDoS_TBLG(1, thetarad)
    P = (12/(4*np.pi**2)) * (1/(np.sqrt(np.abs(alpha*beta))))
    Estar = 0.8*Ev
    DoS = np.array([np.exp(-mu/Estar) * A * np.abs(mu) + (1-np.exp(-mu/Estar)) * P *  np.log(np.abs((BW-Ev)/(mu-Ev))) for mu in Eset])
    TOWRITE_Y = np.concatenate((np.flip(DoS),DoS))
    filename = os.path.join(BASE_PATH,str(ival)+'.csv')
    pathlogfile = os.path.join(BASE_PATH,str(ival)+'log.txt')
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(zip(TOWRITE_X,TOWRITE_Y))
    with open(pathlogfile,'w', newline = '') as f:
        f.write(str((int(thetai*100)/100,Ev)))

### The magic angle case is handled separately. This is where beta becomes zero, and numerically $E_v$ becomes $10^{-7}$, which we assume is zero. The numbers $\gamma $ and  $\kappa$ are computed. These three numbers determine the numbers in the power law DoS. 

#### The ambiguity now is how to continue the tail. The Yuan dispersion is valid until an energy scale of about 1 meV. Then I've extended it to the band edge of 3eV by the logarithm which has the same DoS at mtch = 1meV 

In [ ]:
BASE_PATH = "C:/Users/Aravi/OneDrive/Documents/DoS_Pass2/"
if not os.path.exists(BASE_PATH):
     raise(Exception("CHECK THE BASE PATH"))

BW = 3.0 #consider a rough estimate of the bandwidth to be 3eV
Eset = BW*np.logspace(-12,0,13*10000)
thetai = 1.05


gamma,kappa = np.load('gammakappa.npy')
alpha, beta = np.load('alphabeta.npy')[5]
alpha = 0.5*alpha
Gammatwidsquare = gamma**2 - 4*alpha*kappa 
pref = ((2*np.pi)**(-2.5)) * (GAMMA(0.25)**2) 
denom = (4.0*alpha*Gammatwidsquare)**0.25
C = 12*pref/denom 
mtch = 0.001
G = (C*(mtch**-0.25))/(np.log(BW/mtch))
MagicDos = (np.exp(-Eset/mtch) * C * Eset**(-0.25)) + (1-np.exp(-Eset/mtch))*G*np.log(BW/Eset)

TOWRITE_X = np.concatenate((-np.flip(Eset),Eset))
TOWRITE_Y = np.concatenate((np.flip(MagicDos),MagicDos))

filename = os.path.join(BASE_PATH,'MAGIC'+'.csv')
pathlogfile = os.path.join(BASE_PATH,'MAGIC'+'log.txt')
with open(filename, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(zip(TOWRITE_X,TOWRITE_Y))
with open(pathlogfile,'w', newline = '') as f:
    f.write(str((1.05,0)))